In [1]:
import os.path
import shutil

import numpy as np
import pandas as pd

df = pd.read_csv("data resources/sGSL-scallop-RV-FGP.csv", encoding="windows-1252")
df.columns

Index(['year__annee', 'month__mois', 'day__jour',
       'start_hour__heure_de_depart', 'start_minute__minute_de_depart',
       'latitude', 'longitude', 'gear__equipement', 'species__espece',
       'french_name__nom_francais', 'english_name__nom_anglais',
       'latin_name__nom_latin', 'weight_caught__poids_pris',
       'number_caught__quantite_attrape'],
      dtype='object')

In [2]:
# define the function to combine datetime from a row
def combine_date(row):
    year = str(row["year__annee"]).zfill(4)
    month = str(row["month__mois"]).zfill(2)
    day = str(row["day__jour"]).zfill(2)
    hour = row["start_hour__heure_de_depart"]
    minute = row["start_minute__minute_de_depart"]
    if (not isinstance(hour, str) and np.isnan(hour)) or (not isinstance(minute, str) and np.isnan(minute)):
        return f"{year}-{month}-{day}"
    else:
        return f"{year}-{month}-{day} {int(hour)}:{int(minute)}"

In [3]:
# add a new column that is a datetime
df["datetime"] = pd.to_datetime(df.apply(combine_date, axis=1))
df_trunc = df.sort_values("datetime").loc[:, ["year__annee", "latitude", "longitude", "datetime"]]
df_trunc = df_trunc.drop_duplicates()
df_trunc

,year__annee,latitude,longitude,datetime
0,2012,46.249083,-64.331250,2012-06-27 13:57:00
22,2012,46.335917,-64.325500,2012-06-28 07:59:00
31,2012,46.324417,-64.128167,2012-06-28 09:28:00
45,2012,46.308333,-64.108667,2012-06-28 10:14:00
59,2012,46.280917,-64.099417,2012-06-28 11:15:00
...,...,...,...,...
11484,2022,46.578333,-64.448167,2022-10-18 08:55:00
11504,2022,46.585250,-64.467250,2022-10-18 09:23:00
11514,2022,46.593750,-64.483833,2022-10-18 10:02:00
11518,2022,46.594917,-64.404083,2022-10-18 10:42:00


In [4]:
# Seems like there are some bad points. Let's have a closer look
mask = (df_trunc["latitude"] > 50) | (df_trunc["longitude"] > -60)
df_trunc[mask]


,year__annee,latitude,longitude,datetime


In [5]:
# let's remove those rows
df_trunc = df_trunc[~mask]
df_trunc

,year__annee,latitude,longitude,datetime
0,2012,46.249083,-64.331250,2012-06-27 13:57:00
22,2012,46.335917,-64.325500,2012-06-28 07:59:00
31,2012,46.324417,-64.128167,2012-06-28 09:28:00
45,2012,46.308333,-64.108667,2012-06-28 10:14:00
59,2012,46.280917,-64.099417,2012-06-28 11:15:00
...,...,...,...,...
11484,2022,46.578333,-64.448167,2022-10-18 08:55:00
11504,2022,46.585250,-64.467250,2022-10-18 09:23:00
11514,2022,46.593750,-64.483833,2022-10-18 10:02:00
11518,2022,46.594917,-64.404083,2022-10-18 10:42:00


In [6]:
from stock_assessment_surveys.utils.xy2line import xy2line
xy2line(df_trunc)